In [0]:
WITH base_accounts AS (
  SELECT
    a.acc_key,
    a.acctp_key,
    a.org_key as original_org_key,
    COALESCE(l.loc_geo_latitude, 0) as lat,
    COALESCE(l.loc_geo_longitude, 0) as lon,
    COALESCE(l.loc_geo_jtsk_x, 0) as jtsk_x,
    COALESCE(l.loc_geo_jtsk_y, 0) as jtsk_y,
    -- Determine search radius based on Prague coordinates
    CASE 
      WHEN COALESCE(l.loc_geo_latitude, 0) BETWEEN 49.9 AND 50.2 
       AND COALESCE(l.loc_geo_longitude, 0) BETWEEN 14.1 AND 14.7 
      THEN 0.025  -- Prague: ~1-2 km
      ELSE 0.2    -- Other: ~10 km
    END as search_radius
  FROM gap_catalog.dwh_owner.all_accounts a
  JOIN gap_catalog.dwh_owner.organizations o
    ON a.org_key = o.org_key
  LEFT JOIN gap_catalog.dwh_owner.locations l
    ON o.loc_key = l.loc_key
   AND o.loc_source_system_id = l.loc_source_system_id
   AND l.loc_source_system_id = 'AD0'
  WHERE a.org_key > 0
    AND a.acctp_key NOT IN (9999)
    AND (COALESCE(l.loc_geo_latitude, 0) != 0 OR COALESCE(l.loc_geo_longitude, 0) != 0)
),
nearest_orgs AS (
  SELECT
    ba.acc_key,
    ba.acctp_key,
    so.org_key,
    -- Calculate Euclidean distance
    ABS(SQRT(
      (ba.jtsk_x - so.geo_jtsk_x) * (ba.jtsk_x - so.geo_jtsk_x) + 
      (ba.jtsk_y - so.geo_jtsk_y) * (ba.jtsk_y - so.geo_jtsk_y)
    )) as distance,
    ROW_NUMBER() OVER (
      PARTITION BY ba.acc_key 
      ORDER BY ABS(SQRT(
        (ba.jtsk_x - so.geo_jtsk_x) * (ba.jtsk_x - so.geo_jtsk_x) + 
        (ba.jtsk_y - so.geo_jtsk_y) * (ba.jtsk_y - so.geo_jtsk_y)
      )), so.org_key
    ) as rn
  FROM base_accounts ba
  JOIN gap_catalog.etl_owner.stg_hbr_org so
    ON so.geo_ref_flag = 'Y'
   -- Geographic bounding box filter for performance
   AND so.geo_lat BETWEEN ba.lat - ba.search_radius AND ba.lat + ba.search_radius
   AND so.geo_lon BETWEEN ba.lon - ba.search_radius AND ba.lon + ba.search_radius
)
SELECT
  acc.acc_key,
  acc.acctp_key,
  acc.org_key
FROM (
  SELECT
    ba.acc_key,
    ba.acctp_key,
    COALESCE(no.org_key, -1) as org_key
  FROM base_accounts ba
  LEFT JOIN nearest_orgs no
    ON ba.acc_key = no.acc_key
   AND no.rn = 1
) acc
JOIN gap_catalog.dwh_owner.organizations org
  ON acc.org_key = org.org_key
 AND acc.org_key > 0;